# README #

Bu dosyada data_preparation.ipynb'den çıkan raw_training_data.csv'yi finalize ederek train ve test olarak ikiye bölüyoruz.

- 2013-2014 datasını tablodan çıkarıyoruz çünkü birçok 'time-series' feature'u kullandık ve 2013-2014'te bu değerlerin çoğu boş geliyor.
- Raw data'daki takım ismi, hakem ismi gibi kolonları çıkarttık. Elimizde sadece modelde kullanacağımız feature'lar ve TARGET alanı kaldı
- Null Treatment yaptık:
    - Null treatment'ta, her bir kolon için, o kolonun ortalamasını ve standart sapmasını hesapladık. Sonra her bir boş hücreye ortalama +- standart sapma aralığından rastgele bir değer seçerek o değeri atadık. Böylece randomizasyonu sağlamış ve mantıksız değerler üretmemiş olduk.
    - Sonuç olarak, elimize her hücresi dolu bir data geçti.
- Modellemeye geçmeden önceki son adım train ve test datası olarak iki data oluşturmak. Modeli train'i kullanarak kuracağız ve test'i kullanarak hiç görmediğimiz bir datayı tahmin edip doğru tahmin ettik mi diye bakacağız.
    - train_test_split(raw,test_size=0.2,random_state=17) fonksiyonunu kullanarak raw datayı %80'i train %20'si test olacak şekilde ayırdık ve training_test_data klasöründe train.csv ve test.csv olarak iki dosyaya yazdık.
    
Buraya kadar okuduysanız model_selection.ipynb dosyasına geçebilirsiniz.

In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [7]:
# We read the raw training data extracted from data_preparation.ipynb
raw = pd.read_csv('./raw_training_data/raw_training_data.csv')
cols = list(raw.columns)
# We reduce the columns to only ones that we will use for modeling, e.g. we do not need to include the teams' names in training data.
cols = cols[cols.index('TARGET'):]
raw = raw[cols]
# Since we will use a lot of time-series features, we needed to delete the first season of the data as a lot of the columns are NULL.
raw = raw[raw.Season>20132014]
# At the end, we are only left with TARGET column and the set of features we will use for our modeling.

In [9]:
# More features defined here..
raw['MKT_VAL_RATIO']=raw.HOME_MKT_VAL/raw.AWAY_MKT_VAL

In [10]:
# Null treatment - We produce random numbers that are obtained from a uniform distribution that has a range of mean +- standard deviation of that specific column
pd.options.mode.chained_assignment = None
def null_treat(df, col):
    avg = df[col].mean()
    std = df[col].std()
    null_count = df[col].isnull().sum()
    null_random_list = np.random.uniform(avg - std, avg + std, size=null_count)
    df[col][np.isnan(df[col])] = null_random_list
    return df[col]

In [11]:
for feat in [e for e in raw if e!='TARGET']:
    raw[feat]=null_treat(raw,feat)

In [29]:
raw.drop('Season',inplace=True,axis=1)

In [30]:
splitted = train_test_split(raw,test_size=0.2,random_state=17)

In [31]:
train = pd.DataFrame(splitted[0])
test = pd.DataFrame(splitted[1])

In [32]:
print('Train target mean: '+str(train['TARGET'].mean()))
print('Test target mean: '+ str(test['TARGET'].mean()))
# We made sure that the data is stratified when splitting into train and test.

Train target mean: 1.852796052631579
Test target mean: 1.7960526315789473


In [33]:
train.to_csv('./training_test_data/train.csv',index=False)

In [34]:
test.to_csv('./training_test_data/test.csv',index=False)